In [1]:
import random
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sklearn.datasets import load_boston
import warnings
import joblib

warnings.filterwarnings("ignore")
sys.path.insert(0,'..')

from BMR.ml_models import *

random.seed(0)
np.random.seed(0)

In [2]:
boston = load_boston()

X = boston.data
# drop categorical variables
X = X[:, [0, 1, 2, 4, 5, 6, 7, 9, 10, 11, 12]]
y = boston.target.reshape(-1,1)

In [ ]:
run_all(X, y, f'Boston.joblib', mcloops=10)

*** Method=bmr1 loop=0
*** Method=bmr1 loop=1
*** Method=bmr1 loop=2
*** Method=bmr1 loop=3
*** Method=bmr1 loop=4
*** Method=bmr1 loop=5
*** Method=bmr1 loop=6
*** Method=bmr1 loop=7
*** Method=bmr1 loop=8
*** Method=bmr1 loop=9
*** Method=bmr2 loop=0
*** Method=bmr2 loop=1
*** Method=bmr2 loop=2
*** Method=bmr2 loop=3
*** Method=bmr2 loop=4
*** Method=bmr2 loop=5
*** Method=bmr2 loop=6
*** Method=bmr2 loop=7
*** Method=bmr2 loop=8
*** Method=bmr2 loop=9
*** Method=bmr3 loop=0
*** Method=bmr3 loop=1
*** Method=bmr3 loop=2
*** Method=bmr3 loop=3
*** Method=bmr3 loop=4
*** Method=bmr3 loop=5
*** Method=bmr3 loop=6
*** Method=bmr3 loop=7
